# Challenge 3

In this challenge we will work on the `Orders.csv` data set in the previous [Subsetting and Descriptive Stats lab (../../lab-subsetting-and-descriptive-stats/your-code/main.ipynb). In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

# Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [2]:
# import required libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

Next, import `Orders.csv` from the "subsetting" lab folder into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [3]:
#The csv does not exist, we connect to the DB
#Connect to DB
driver = 'mysql+pymysql:'
user = 'ironhacker_read'
password = 'ir0nhack3r'
ip = '35.239.232.23'
database = 'orders'

connection_string = f'{driver}//{user}:{password}@{ip}/{database}'
engine = create_engine(connection_string)

In [6]:
orders = pd.read_sql('orders',connection_string)
orders.head()

,index,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [27]:
# your code here
# Sub-problem 1: Group by customer and order by sum of amount spent
orders_bycustomer = orders.groupby('CustomerID').sum().sort_values('amount_spent', ascending = False)
orders_bycustomer.head()

,index,InvoiceNo,year,month,day,hour,Quantity,UnitPrice,amount_spent
CustomerID,,,,,,,,,
14646,386260126,1163267611,4182810,14191,6552,24488,197491,5176.09,280206.02
18102,98902435,243297801,866723,3746,1261,5587,64124,1940.92,259657.30
17450,66203333,188845149,677704,2292,842,4140,69993,1143.32,194550.79
16446,664956,1688629,6033,22,11,27,80997,4.98,168472.50
14911,1245078456,3196374868,11416155,46220,18930,68148,80515,26185.72,143825.06


In [46]:
#Sub-problem 2: select customers whose aggregated amount_spent is in a given quantile range?
total_spent = sum(orders['amount_spent'])
orders_bycustomer['rel_amount_spent'] = (orders_bycustomer['amount_spent']/total_spent)*100
orders_bycustomer['rel_amount_spent_cum'] = orders_bycustomer['rel_amount_spent'].cumsum()
orders_bycustomer.head()
#Given a quartile, eg. 25%
quartile = 25
orders_bycustomer[orders_bycustomer['rel_amount_spent_cum'] < quartile]

,index,InvoiceNo,year,month,day,hour,Quantity,UnitPrice,amount_spent,rel_amount_spent,rel_amount_spent_cum
CustomerID,,,,,,,,,,,
14646,386260126,1163267611,4182810,14191,6552,24488,197491,5176.09,280206.02,3.144352,3.144352
18102,98902435,243297801,866723,3746,1261,5587,64124,1940.92,259657.30,2.913763,6.058115
17450,66203333,188845149,677704,2292,842,4140,69993,1143.32,194550.79,2.183166,8.241280
16446,664956,1688629,6033,22,11,27,80997,4.98,168472.50,1.890526,10.131807
14911,1245078456,3196374868,11416155,46220,18930,68148,80515,26185.72,143825.06,1.613943,11.745750
12415,114857574,398543981,1439876,4254,2169,8061,77670,2097.08,124914.53,1.401737,13.147487
14156,235637584,780541003,2815397,8630,4284,17201,57885,5357.68,117379.63,1.317184,14.464671
17511,176975396,538173446,1936505,7215,2701,11320,64549,2221.28,91062.38,1.021863,15.486534
16029,38597667,134645942,486630,1777,748,2789,40208,8722.39,81024.84,0.909226,16.395760


In [52]:
#Sub-problem 3: Label the VIPs will be the ones in the 10ile and the preferred from the 10ile+ to the 25ile
def VIP_or_preferred(x):
    if x  < 10:
        return "VIP"
    elif x > 10 and x < 25:
        return "Preferred"
    else:
        return "Normal" 
    
orders_bycustomer['customer_status'] = orders_bycustomer['rel_amount_spent_cum'].apply(VIP_or_preferred)
orders_bycustomer.head(15)

,index,InvoiceNo,year,month,day,hour,Quantity,UnitPrice,amount_spent,rel_amount_spent,rel_amount_spent_cum,customer_status
CustomerID,,,,,,,,,,,,
14646,386260126,1163267611,4182810,14191,6552,24488,197491,5176.09,280206.02,3.144352,3.144352,VIP
18102,98902435,243297801,866723,3746,1261,5587,64124,1940.92,259657.30,2.913763,6.058115,VIP
17450,66203333,188845149,677704,2292,842,4140,69993,1143.32,194550.79,2.183166,8.241280,VIP
16446,664956,1688629,6033,22,11,27,80997,4.98,168472.50,1.890526,10.131807,Preferred
14911,1245078456,3196374868,11416155,46220,18930,68148,80515,26185.72,143825.06,1.613943,11.745750,Preferred
12415,114857574,398543981,1439876,4254,2169,8061,77670,2097.08,124914.53,1.401737,13.147487,Preferred
14156,235637584,780541003,2815397,8630,4284,17201,57885,5357.68,117379.63,1.317184,14.464671,Preferred
17511,176975396,538173446,1936505,7215,2701,11320,64549,2221.28,91062.38,1.021863,15.486534,Preferred
16029,38597667,134645942,486630,1777,748,2789,40208,8722.39,81024.84,0.909226,16.395760,Preferred


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

# Q2: How to identify which country has the most VIP Customers?

# Q3: How to identify which country has the most VIP+Preferred Customers combined?

Provide your solution for Q2 below:

In [59]:
# We add the customer status info to the main DF
orders_status = orders.merge(orders_bycustomer[['customer_status']],how = 'inner',left_on = 'CustomerID', right_index = True)

In [72]:
#Group by Country
orders_status.groupby(['CustomerID','Country','customer_status']).count()

,,,index,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,amount_spent
CustomerID,Country,customer_status,,,,,,,,,,,,
12346,United Kingdom,Preferred,1,1,1,1,1,1,1,1,1,1,1,1
12347,Iceland,Normal,182,182,182,182,182,182,182,182,182,182,182,182
12348,Finland,Normal,31,31,31,31,31,31,31,31,31,31,31,31
12349,Italy,Normal,73,73,73,73,73,73,73,73,73,73,73,73
12350,Norway,Normal,17,17,17,17,17,17,17,17,17,17,17,17
12352,Norway,Normal,85,85,85,85,85,85,85,85,85,85,85,85
12353,Bahrain,Normal,4,4,4,4,4,4,4,4,4,4,4,4
12354,Spain,Normal,58,58,58,58,58,58,58,58,58,58,58,58
12355,Bahrain,Normal,13,13,13,13,13,13,13,13,13,13,13,13
